# ETL_ CREDITS
TRANSFORMACIONES: 
- Eliminación de campos innecesarios en el análisis.<P>
- Cambio de nombres.<P>
- Cambio de tipos de datos.<P>
- Verificar los valores nulos del campo. <P>
- Eliminación de valores duplicados.<P>

In [40]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast
import sys
import os

# Agrega el directorio superior al path
sys.path.append(os.path.abspath(".."))

# Importa la función desanidar_concat
from funciones.desanidar_concatena import desanidar_concat
from funciones.reordena_col import reordena_colum


### 0.1. Carga de los datos

In [63]:
credits_test  = pd.read_csv('../datasets_original/credits.csv', sep=',',quotechar='"', encoding='utf-8' )

In [64]:
print(credits_test.shape)

(45476, 3)


Cambio de nombre columna 'id'=> 'IdMovie'

In [65]:
credits_test.rename(columns={'id':'id_movie'}, inplace=True)

Se verifica

In [22]:
credits_test.head(1)

,cast,crew,id_movie
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


### 0.2. Se obtiene dataframe CREW y dataframe CAST

In [66]:
# Se asignan a nuevos dataframes y se eliminan columnas no necesarias en cada uno
cast_df = credits_test.drop	(columns = "crew")
crew_df = credits_test.drop(columns = "cast")

In [67]:
type(cast_df)

pandas.core.frame.DataFrame

In [68]:
# se ordenan columnas, con Id como primera
cast_df = cast_df[['id_movie', 'cast']]
crew_df= crew_df[['id_movie', 'crew']]

In [69]:
# Se verifica
cast_df.head(1)

,id_movie,cast
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',..."


In [70]:
crew_df.head(1)

,id_movie,crew
0,862,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


Desanidar cast y crew:

In [71]:
# Convertimos las cadenas de texto a listas de diccionarios
cast_df['cast'] = cast_df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Usamos explode para separar cada diccionario en una fila
credits_exploded = cast_df.explode('cast').reset_index(drop=True)

# Desanidar cada diccionario en columnas individuales usando json_normalize
cast_df = pd.json_normalize(credits_exploded['cast'])

# Agregamos el 'id_movie' al nuevo DataFrame
cast_df['id_movie'] = credits_exploded['id_movie']

cast_df.head(2)

,cast_id,character,credit_id,gender,id,name,order,profile_path,id_movie
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862


In [72]:
# Conviertiendo las cadenas en la columna 'crew' a listas de diccionarios.
crew_df['crew'] = crew_df['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Desanidando la columna 'crew' (explode) creando una fila por cada elemento en la lista
data_credits_exploded = crew_df.explode('crew')

# Restablezco el indice por problemas con duplicados despues de desanidar
data_credits_exploded = data_credits_exploded.reset_index(drop=True)

# Normalizo los diccionarios dentro de la columna 'crew' en columnas separadas
data_credits_crew = pd.json_normalize(data_credits_exploded['crew'])

# Mantengo la columna 'id' de la pelicula en el dataset normalizado
data_credits_crew['id_movie'] = data_credits_exploded['id_movie']

# Filtrado de las filas que tengan 'Director'
crew_df = data_credits_crew[data_credits_crew['job'] == 'Director']

crew_df.head(2)

,credit_id,department,gender,id,job,name,profile_path,id_movie
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
109,52fe44bfc3a36847f80a7c7d,Directing,2.0,4945.0,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg,8844


## 1. TRANSFORMACIONES

In [73]:
cast_df.head(2)

,cast_id,character,credit_id,gender,id,name,order,profile_path,id_movie
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862


### Eliminación de columnas

COLUMNAS:
- credit_id_cast, gender_cast, order_cast, profile_path_cast

In [74]:
columnas_cast = [ 'character','credit_id',  'order', 'profile_path']
columnas_crew = [ 'credit_id', 'department', 'profile_path']

cast_df = cast_df.drop(columnas_cast, axis=1) 
crew_df = crew_df.drop(columnas_crew, axis=1) 


In [75]:
crew_df.head(2)

,gender,id,job,name,id_movie
0,2.0,7879.0,Director,John Lasseter,862
109,2.0,4945.0,Director,Joe Johnston,8844


### Valores nulos

In [76]:
print(cast_df.shape)
cast_df.isnull().sum()

(564892, 5)


cast_id     2418
gender      2418
id          2418
name        2418
id_movie       0
dtype: int64

In [77]:
print(crew_df.shape)
crew_df.isnull().sum()

(49048, 5)


gender      0
id          0
job         0
name        0
id_movie    0
dtype: int64

In [78]:
# Se elimina solo los valores nulos
columns_cast = ['cast_id', 'gender', 'id', 'name']

# Elimina las filas con valores nulos 
cast_df = cast_df.dropna(subset=columns_cast)

### Cambiar nombre y tipo de dato

In [79]:
crew_df.head(1)

,gender,id,job,name,id_movie
0,2.0,7879.0,Director,John Lasseter,862


Cambio de nombre columnas

In [80]:
columnas_new_cast = {'cast_id':'id_cast', 'gender':'gender_actor', 
            'id':'id_actor','name':'name_actor'}

columnas_new_crew = {'gender':'gender_dir', 'id':'id_dir', 
            'name':'name_dir'}

cast_df.rename(columns=columnas_new_cast, inplace=True)
crew_df.rename(columns=columnas_new_crew, inplace=True)

In [81]:
# Reorganiza columnas
cast_df= reordena_colum(cast_df, 'id_movie',0)
crew_df= reordena_colum(crew_df, 'id_movie',0)
crew_df= reordena_colum(crew_df, 'id_dir',1)


Tipo de datos

In [82]:
cast_df.dtypes

id_movie          int64
id_cast         float64
gender_actor    float64
id_actor        float64
name_actor       object
dtype: object

In [83]:
crew_df.dtypes

id_movie        int64
id_dir        float64
gender_dir    float64
job            object
name_dir       object
dtype: object

In [87]:
cast_df.head()

,id_movie,id_cast,gender_actor,id_actor,name_actor
0,862,14,2,31,Tom Hanks
1,862,15,2,12898,Tim Allen
2,862,16,2,7167,Don Rickles
3,862,17,2,12899,Jim Varney
4,862,18,2,12900,Wallace Shawn


In [88]:
crew_df.head()

,id_movie,id_dir,gender_dir,job,name_dir
0,862,7879,2,Director,John Lasseter
109,8844,4945,2,Director,Joe Johnston
122,15602,26502,2,Director,Howard Deutch
126,31357,2178,2,Director,Forest Whitaker
141,11862,56106,2,Director,Charles Shyer


In [86]:
cast_df["id_cast"] = cast_df["id_cast"].astype('Int64')
cast_df["gender_actor"] = cast_df["gender_actor"].astype('Int64')
cast_df["id_actor"] = cast_df["id_actor"].astype('Int64')

crew_df["id_dir"] = crew_df["id_dir"].astype('Int64')
crew_df["gender_dir"] = crew_df["gender_dir"].astype('Int64')

COLUMNAS:
- credit_id_crew, gender_crew, profile_path_crew

In [32]:
columnas = [ 'credit_id_crew', 'gender_crew', 'profile_path_crew']
crew_df = crew_df.drop(columnas, axis=1) 

In [33]:
crew_df.head(1)

,IdMovie,department_crew,id_crew,job_crew,name_crew
0,862,Directing,7879,Director,John Lasseter


## 3. EXPORTACIÓN DATASET FINAL 

In [90]:
print(crew_df.shape)
crew_df.head(2)

(49048, 5)


,id_movie,id_dir,gender_dir,job,name_dir
0,862,7879,2,Director,John Lasseter
109,8844,4945,2,Director,Joe Johnston


In [91]:
print(cast_df.shape)
cast_df.head(2)

(562474, 5)


,id_movie,id_cast,gender_actor,id_actor,name_actor
0,862,14,2,31,Tom Hanks
1,862,15,2,12898,Tim Allen


In [93]:
# Exportación
crew_df.to_parquet('../datasets/credits_crew.parquet', engine='pyarrow')
cast_df.to_parquet('../datasets/credits_cast.parquet', engine='pyarrow')